# **CE272 - Programming Assignment**
### **Traffic Network Equilibrium**

#### **Name**: Debojjal Bagchi
#### **SRN** : 16849
#### **Course** : Undergraduate


---









# **Filename & Import Libraries**




In [35]:
!git clone https://github.com/bstabler/TransportationNetworks.git

fatal: destination path 'TransportationNetworks' already exists and is not an empty directory.


In [36]:
#Network Filename

filename = "/content/TransportationNetworks/SiouxFalls/SiouxFalls_net.tntp"
#filename = "/content/TransportationNetworks/Eastern-Massachusetts/EMA_net.tntp"
#filename = "/content/TransportationNetworks/Anaheim/Anaheim_net.tntp"
#filename = "/content/TransportationNetworks/Chicago-Sketch/ChicagoSketch_net.tntp"

#Demand Filename

demand_filename = "/content/TransportationNetworks/SiouxFalls/SiouxFalls_trips.tntp"
#demand_filename = "/content/TransportationNetworks/Eastern-Massachusetts/EMA_trips.tntp"
#demand_filename = "/content/TransportationNetworks/Anaheim/Anaheim_trips.tntp"
#demand_filename = "/content/TransportationNetworks/Chicago-Sketch/ChicagoSketch_trips.tntp"



In [37]:
import numpy as np
import operator
from operator import add
import functools
import time

# **Programming Task 1 : Preliminaries**

### **Define various classes**





*   Class Node 
      *   Node Number
      *   Adjacency List


*   Class Arc
      *  All Atributes



*   Node Array - Contains the list of Nodes
*   Arc Array - Contains the list of Arcs
*italicized text*












In [38]:
class Node:
  def __init__ (self, node_number):
    self.node_number = node_number
    self.adjacency_list = []
    self.adjacent_arc_list = []


class Arc :
  def __init__ (self, tail, head):
    self.tail = tail
    self.head = head
    self.capacity = 0
    self.length = 0
    self.free_flow_time = 0
    self.B = 0
    self.power = 0
    self.speed_limit = 0
    self.toll = 0
    self.types = 0
    self.flow = 0
    self.aon_flow = 0
    self.time = 0
    


Node_Array = []
Arc_Array = []

### **Read Data from the input file**

In [39]:
#Open The Net.tntp File
with open(filename, "r") as f:

#Deal with the metadata. 
  count =0
  for line in f:
      line = line.rstrip()
      #Parse the first line & Assign NumZone as the Number of Zones 
      if count == 0:
        line = line.split('>')
        NumZone = int(line[1])

      #Parse the Second line & Assign NumNode as the Number of Nodes
      if count == 1:
        line = line.split('>')
        NumNode = int(line[1])

      #Parse the Third line & Assign FirstThruNode 
      if count == 2:
        line = line.split('>')
        FirstThruNode = int(line[1])

      #Parse the Fourth line & Assign Numlinks as the Number of Links
      if count == 3:
        line = line.split('>')
        Numlink = int(line[1])
      

      #Our Real Data starts from the line containing ~
      count+=1

      if str(line).find('~') == 0:
        break


#Parse the data from the input file into Node Class & Arc Class. 
#Each line correspnds to a node class & arc class. \
#The arc classes are fine but the node class is not OK as each line is not a node. 
#This is dealt in next snippet.

  i = 0
  for line in f:
      if len(line.strip())==0:
        continue
      line = line.rstrip()
      line = line.split(";")[0].split('\t')
      #print([line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9],line[10]])

      Node_Array.append(Node(0))
      Node_Array[i].node_number = int(line[1])
      Node_Array[i].adjacency_list = [int(line[2])]
      Node_Array[i].adjacent_arc_list = [i+1]

      Arc_Array.append(Arc(0,0))
      Arc_Array[i].tail = int(line[1])
      Arc_Array[i].head = int(line[2])
      Arc_Array[i].capacity = float(line[3])
      Arc_Array[i].length = float(line[4])
      Arc_Array[i].free_flow_time = float(line[5])
      Arc_Array[i].B = float(line[6])
      Arc_Array[i].power = float(line[7])
      Arc_Array[i].speed_limit = float(line[8])
      Arc_Array[i].toll = float(line[9])
      Arc_Array[i].types = float(line[10])

      #Uncomment the next line to check if loaded data is okay
      #print([line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9],line[10]])


      i+=1


In [40]:
#Fixing the node class such that there are NumNode number of Node Classes.
#This is done by comparing two node class, if the have same Node_Number then
#Adjacency list of the two nodes are merged.
#This is done for all Nodes itteratively. 
#Note that the Init_Node column is sorted in Data File. Hence one loop does the job!
Node_Array.sort(key=operator.attrgetter('node_number'))

i = i-1
while i > 0 :
  if Node_Array[i].node_number == Node_Array[i-1].node_number:
     Node_Array[i-1].adjacency_list.extend(Node_Array[i].adjacency_list)
     Node_Array[i-1].adjacent_arc_list.extend(Node_Array[i].adjacent_arc_list)
     Node_Array.pop(i)
     
  i=i-1


In [41]:
print("length of Node Array=",len(Node_Array))

length of Node Array= 24


This loop will fail when the tail nodes are not in order- check if it will work for all networks.else sort it. 
Also, the pop() doesnt seem to work ( look at the length of Node_Array)

**Re**:
1. I have sorted the array.
2. Can you explain me breifly why pop() wont work. Did not understand that part.


### **Define a function to print Node Node Ajacency Matrix**

In [42]:
def Show_NNAM(p,q):
  rows, cols = (NumNode+1, NumNode+1)
  frmt = "{:>3}"*(cols)

  Node_Node_Adj_Matrix = [[0 for i in range(cols)] for j in range(rows)]

  i = 0


  while i < NumNode :
    for j in Node_Array[i].adjacency_list:
      Node_Node_Adj_Matrix[Node_Array[i].node_number][j] = 1
    i+=1

  for i in range(NumNode+1):
    Node_Node_Adj_Matrix[i][i] = 1
    Node_Node_Adj_Matrix[i][0] = i
    Node_Node_Adj_Matrix[0][i] = i


  if p != 0 :
    return(Node_Node_Adj_Matrix[p][q])
 
  if p == 0 :
    for row in Node_Node_Adj_Matrix:
      print(frmt.format(*row))
  
  




### **Define a function to print Node Arc Incidence Matrix**

In [43]:
def Show_NAIM(p,q):
  rows, cols = (NumNode+1, Numlink+1)
  frmt = "{:>3}"*(cols)

  Node_Arc_Inc_Matrix = [[0 for i in range(cols)] for j in range(rows)]

  i = 0 
  
  while i < Numlink :
    Node_Arc_Inc_Matrix[0][i+1] = i+1
    Node_Arc_Inc_Matrix[Arc_Array[i].tail ][i+1] = -1
    Node_Arc_Inc_Matrix[Arc_Array[i].head][i+1] = 1

    i+=1
 
  i=0
  while i <= NumNode :
    Node_Arc_Inc_Matrix[i][0] = i
    i+=1

  
  if p != 0 :
    return(Node_Arc_Inc_Matrix[p][q])
 
  if p == 0 :
    for row in Node_Arc_Inc_Matrix:
      print(frmt.format(*row))



 

### **Define a function to print Node Ajacency List**

In [44]:
def Show_AL(p):

  Adj_List = {}

  i=0

  while i< NumNode :
    Adj_List[Node_Array[i].node_number] =  Node_Array[i].adjacency_list
    i+=1

  if p !=0 :
     return(Adj_List[p])

  if p == 0: 
    for x, y in Adj_List.items():
      print(x, y)



### **Print Outputs**

#### **Instructions for Node Node Adjacency Matrix**



* To efficiently print whole matrix use:
```
Show_NNAM(0,0)
```

* To Print the Value of Matrix at any (i,j) use:
```
Show_NNAM(i,j)
```




In [45]:
Show_NNAM(0,0)

  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  2  1  1  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  3  1  0  1  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
  4  0  0  1  1  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
  5  0  0  0  1  1  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  6  0  1  0  0  1  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  7  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
  8  0  0  0  0  0  1  1  1  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
  9  0  0  0  0  1  0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 10  0  0  0  0  0  0  0  0  1  1  1  0  0  0  1  1  1  0  0  0  0  0  0  0
 11  0  0  0  1  0  0  0  0  0  1  1  1  0  1  0  0  0  0  0  0  0  0  0  0
 12  0  0  1  0  0  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0
 13  0  0  0



---


#### **Instructions for Node Arc Adjacency Matrix**


* To efficiently print whole matrix use:
```
Show_NAIM(0,0)
```

* To Print the Value of Matrix at any (i,j) use:
```
Show_NAIM(i,j)
```


In [46]:
Show_NAIM(0,0)

  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76
  1 -1 -1  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  2  1  0 -1 -1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  3  0  1  0  0 -1 -1 -1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  4  0  0  0  0  0  1  0 -1 -1 -1  1  0  0  0  0  0  0  0  0  0  0  0  0



---

#### **Instructions for Node Arc Adjacency Matrix**




* To efficiently print whole List use:
```
Show_AL(0)
```

* To Print the Value of Adjacency List at any node (i) use:
```
Show_AL(i)
```


In [47]:
Show_AL(0)

1 [2, 3]
2 [1, 6]
3 [1, 4, 12]
4 [3, 5, 11]
5 [4, 6, 9]
6 [2, 5, 8]
7 [8, 18]
8 [6, 7, 9, 16]
9 [5, 8, 10]
10 [9, 11, 15, 16, 17]
11 [4, 10, 12, 14]
12 [3, 11, 13]
13 [12, 24]
14 [11, 15, 23]
15 [10, 14, 19, 22]
16 [8, 10, 17, 18]
17 [10, 16, 19]
18 [7, 16, 20]
19 [15, 17, 20]
20 [18, 19, 21, 22]
21 [20, 22, 24]
22 [15, 20, 21, 23]
23 [14, 22, 24]
24 [13, 21, 23]


# **Programming Task 2 : One to All Shortest Path Algorithm**


### **Redefine the class for the one to all shortest path Algorithms**

In [48]:
class Node:
  def __init__ (self, node_number):
    self.node_number = node_number
    self.adjacency_list = []
    self.adjacent_arc_list = []
    self.distance_label
    self.pred_node
    self.pred_arc

In [49]:
inf = float('inf')

### **Define aij matrix.**

In [50]:
#aij
rows, cols = (NumNode+1, NumNode+1)
frmt = "{:>15}"*(cols)

aij = [[inf for i in range(cols)] for j in range(rows)]

i = 0
while i < Numlink :
  aij[Arc_Array[i].tail][Arc_Array[i].head] = Arc_Array[i]
  i+=1

for i in range(NumNode+1):
  aij[i][0] = np.nan
  aij[0][i] = np.nan


### **Define `label_correcting(origin_node)` function**

In [51]:
def label_correcting(origin_node,data):
  #Step1: Initialise
  mu=[]
  pi=[]
  for i in range(NumNode+1):
    mu.append(inf)
    pi.append(-1)
  
  mu[origin_node] = 0
  pi[origin_node] = origin_node 
  mu[0]= np.nan
  pi[0]= np.nan

  sel =[]
  sel.append(origin_node)   

  #Step2: 

  while len(sel) > 0:
    i=sel[0]
    sel.remove(i)

    #1. Using free floow time
    if data ==1:
      for j in Node_Array[i-1].adjacency_list:
        if j>= FirstThruNode:
          if mu[j] > mu[i] + aij[i][j].free_flow_time :
            mu[j] = mu[i] + aij[i][j].free_flow_time
            pi[j] = i
            if j not in sel :
              sel.append(j)
        else:
          if mu[j] > mu[i] + aij[i][j].free_flow_time :
            mu[j] = mu[i] + aij[i][j].free_flow_time
            pi[j] = i


    #2. Using time
    if data ==2:
      for j in Node_Array[i-1].adjacency_list:
        if j>= FirstThruNode:
          if mu[j] > mu[i] + aij[i][j].time :
            mu[j] = mu[i] + aij[i][j].time
            pi[j] = i
            if j not in sel :
              sel.append(j)
        else:
          if mu[j] > mu[i] + aij[i][j].time :
            mu[j] = mu[i] + aij[i][j].time
            pi[j] = i

          


  #print(sel) #Uncomment to view sel
  return mu,pi




### **Print `label_correcting(origin_node)`**

In [52]:
print(label_correcting(1,1)[0])
print(label_correcting(1,1)[1])

[nan, 0, 6.0, 4.0, 8.0, 10.0, 11.0, 16.0, 13.0, 15.0, 18.0, 14.0, 8.0, 11.0, 18.0, 23.0, 18.0, 20.0, 18.0, 22.0, 22.0, 18.0, 20.0, 17.0, 15.0]
[nan, 1, 1, 1, 3, 4, 2, 8, 6, 5, 9, 4, 3, 12, 11, 14, 8, 16, 7, 17, 18, 24, 21, 24, 13]


### **Define `label_setting(origin_node)` function**

In [53]:
def label_setting(origin_node,data):
  #Step1: Initialise
  mu=[]
  pi=[]
  S=[]
  S_bar=[]

  S_bar.append(1)

  for val in range(1, NumNode+1):
    S_bar.append(val)
  for val in range(NumNode+1):
    mu.append(inf)
    pi.append(-1)
  
  
  mu[origin_node] = 0
  pi[origin_node] = origin_node 
  mu[0]= np.nan
  pi[0]= np.nan
  
  #Step2

  while len(S_bar) > 0: 
    r = inf
    for min_j in S_bar: 
      if r >= mu[min_j] :
        i = min_j  
        r = mu[min_j]      

    S_bar.remove(i)
    S.append(i)

#1. Using free floow time
    if data ==1:
      for j in Node_Array[i-1].adjacency_list :
        if j>= FirstThruNode: #FirstThruNode
          if mu[j] > mu[i] + aij[i][j].free_flow_time :
            mu[j] = mu[i] + aij[i][j].free_flow_time
            pi[j] = i
    
#2. using time 
    if data ==2:
      for j in Node_Array[i-1].adjacency_list :
        if j>= FirstThruNode: #FirstThruNode
          if mu[j] > mu[i] + aij[i][j].time :
            mu[j] = mu[i] + aij[i][j].time
            pi[j] = i

  for i in range(1,NumNode+1):
    for j in Node_Array[i-1].adjacency_list:
      if j < FirstThruNode: #FirstThruNode
        #print(i,j)
        if data ==1 :
          if mu[j] > mu[i] + aij[i][j].free_flow_time :
            mu[j] = mu[i] + aij[i][j].free_flow_time
            pi[j] = i
        if data ==2 :
          if mu[j] > mu[i] + aij[i][j].time :
            mu[j] = mu[i] + aij[i][j].time
            pi[j] = i

  

  #print(S) #Uncommment to view S
  #print(S_bar) #Uncomment to view S_bar
  return mu,pi
  


### **Print `label_setting(origin_node)`**

In [54]:
print(label_setting(1,1)[0])
print(label_setting(1,1)[1])

[nan, 0, 6.0, 4.0, 8.0, 10.0, 11.0, 16.0, 13.0, 15.0, 18.0, 14.0, 8.0, 11.0, 18.0, 23.0, 18.0, 20.0, 18.0, 22.0, 22.0, 18.0, 20.0, 17.0, 15.0]
[nan, 1, 1, 1, 3, 4, 2, 8, 6, 5, 9, 12, 3, 12, 11, 14, 8, 16, 7, 17, 18, 24, 21, 24, 13]


# **Programming Task 3 : The MSA Algorithm**



## **Create` demand_Matrix[i][j]`** 


In [55]:
rows, cols = (NumZone+1, NumZone+1)
frmt = "{:>10}"*(cols)
demand_Matrix = [[0 for i in range(cols)] for j in range(rows)]

for i in range(NumZone+1):
  demand_Matrix[i][0] = 0
  demand_Matrix[0][i] = 0

#Uncomment below to see empty demand Matrix
'''
for row in demand_Matrix:
      print(frmt.format(*row))
'''

'\nfor row in demand_Matrix:\n      print(frmt.format(*row))\n'

## **Read data into `demand_Matrix[i][j]`**

In [56]:
#Open The trips.tntp File
with open(demand_filename, "r") as f:

#Deal with the metadata. 
  count =0
  for line in f:
      line = line.rstrip()
      #Parse the first line & Assign NumZone as the Number of Zones 
      if count == 0:
        line = line.split('>')
        NumZone = int(line[1])
      
      #Our Real Data starts from or after the 5th line
      count+=1
      if count >=5:
        break  

#Parse the data 

  i = 0
  node_number = 0
  for line in f:
      
      #Break each line at the delim. ";"
      if len(line.strip())==0:
        continue
      line = line.rstrip()
      line = line.split(";")


      #Some Files have non required data starting with ~. Ignoring those lines!
      if (str(line).find('~') != -1):
        continue

      #Lines have starting node number has just that.
      if len(line) == 1 :
        node_number += 1

      #The other lines have ending node of that starting node
      if len(line) > 1 :
                
        for demand in line:
          if node_number > NumZone:
            continue
          if len(demand.split(":")) > 1:
            demand_Matrix[node_number][int(demand.split(":")[0])] = demand.split(":")[1]

      i+=1



In [57]:
#Convert string values into float
for i in range(cols) :
  for j in range(rows) :
    demand_Matrix[i][j] = float(demand_Matrix[i][j])



In [58]:
#Uncomment the following to view demand_Matrix[i][j]
'''
for row in demand_Matrix:
      print(frmt.format(*row))
'''

'\nfor row in demand_Matrix:\n      print(frmt.format(*row))\n'

In [59]:
#Appriximation. Use this to get close by TSTT in much lesser time.
#I have reported time in both cases with & without approximation
#If demand matrix is large, we ignore demands between OD Pairs that ar less than 10^-3 times max demand
'''
if len(demand_Matrix) > 300:
  max =0
  for i in range(cols) :
    for j in range(rows) :
      if max < demand_Matrix[i][j] :
        max = demand_Matrix[i][j]
  max

  for i in range(cols) :
    for j in range(rows) :
      if demand_Matrix[i][j] < max/1000:
        demand_Matrix[i][j] = 0
'''

'\nif len(demand_Matrix) > 300:\n  max =0\n  for i in range(cols) :\n    for j in range(rows) :\n      if max < demand_Matrix[i][j] :\n        max = demand_Matrix[i][j]\n  max\n\n  for i in range(cols) :\n    for j in range(rows) :\n      if demand_Matrix[i][j] < max/1000:\n        demand_Matrix[i][j] = 0\n'

## **Define the BPR Function to calculate travel times**

In [60]:
#This function returns ue travel time & so travel time for a given flow at a particular arc

'''
Input : flows
'''

def BPR(i,flow):
  free_flow_time= Arc_Array[i].free_flow_time
  B= Arc_Array[i].B 
  capacity = Arc_Array[i].capacity
  power =  Arc_Array[i].power
  time_ue = free_flow_time*(1+B*(flow[i]/capacity)**power) #UE
  time_so = time_ue + (power * free_flow_time * B *(flow[i]/capacity)**power) #SO

  return time_ue, time_so


In [61]:
#This function updates ue travel time Or so travel time (need to uncomment) for a given flow to the arc class's time

'''
Input : flows
'''

def time_fn(flow):

  i = 0
  while i < Numlink :
    aij[Arc_Array[i].tail][Arc_Array[i].head].time = BPR(i,flow)[0] #UE
    #aij[Arc_Array[i].tail][Arc_Array[i].head].time = BPR(i,flow)[1] #SO
    i+=1

  


## **Call the one to all shortest path algorithms**

In [62]:
#This function updates aon flow to the arc class's aon_flow by assigning required demand to the shotest path 

'''
Input : data = 0 : Uses free flow time to calculate shortest path
        data = 1 : Uses aon flows to calculate shortest path
'''

def shortest_path (data):

  rows = NumZone+1
  for start_node in range(1,rows):
    #label setting
    #mu =  label_setting(start_node,data)[1]
    #label correcting
    mu =  label_correcting(start_node,data)[1]


    for end_node in  range(1,rows):
      destination_node =  end_node 

      if demand_Matrix[start_node][end_node] == 0 or start_node==end_node:
        continue


      while destination_node != start_node:
        aij[mu[destination_node]][destination_node].aon_flow = aij[mu[destination_node]][destination_node].aon_flow + demand_Matrix[start_node][end_node]
        destination_node =  mu[destination_node]



In [63]:
#This function returns the all or nothing flows by calling the shoest path function

'''
Input : data = 0 : Uses free flow time to calculate shortest path
        data = 1 : Uses aon flows to calculate shortest path
'''

def xhat_fn(data):

  x = []

  #Initially set AON Flows as 0
  i = 0
  while i < Numlink :
    aij[Arc_Array[i].tail][Arc_Array[i].head].aon_flow = 0
    i+=1

  #Call the shortest path algorithm
  shortest_path(data)

  #Update aon flows vector
  i = 0
  while i < Numlink :
    x.append(aij[Arc_Array[i].tail][Arc_Array[i].head].aon_flow)
    i+=1
  
  return x


## **Define Convergence Measures : RG, TSTT, SPTT**

In [64]:
#This function works same as sumproduct in excel

'''
Input : *lists : list of whose sumproduct is required
'''

def sumproduct(*lists):
    return sum(functools.reduce(operator.mul, data) for data in zip(*lists))

In [65]:
#This function finally calculates the RG & returns RG, TSTT, SPTT in that order

'''
Input : x, xhat
'''

def calculate_gap(x,xhat):

  tx = []

  for i in range(len(Arc_Array)):
    
    tx.append(BPR(i,x)[0])

  TSTT = sumproduct(x,tx)
  SPTT = sumproduct(tx,xhat)
  RG = TSTT/SPTT - 1 

  return RG,TSTT,SPTT




## **MSA Algorithm**



In [66]:
#Find a feasible xhat. Initialise x with shortest path with free flow travel time
start_time = time.time()
k=1

x =[]
for points in range(Numlink):
  x.append(0) 

time_fn(x)
x = xhat_fn(2)

time_fn(x)
xhat = xhat_fn(2)

#Calculate RG
RG = calculate_gap(x,xhat)

#While RG not converging
while RG[0] > 0.001:
  #Update x as conv. combination
  x = list(map(add, [val * (1/k) for val in xhat],[val * (1-1/k) for val in x]) ) 

  '''
  #Update AON Flows
  while i < Numlink :
      aij[Arc_Array[i].tail][Arc_Array[i].head].aon_flow = x[i]
      i+=1
  '''
  #Update t(x)
  time_fn(x)

  #Update xhat
  xhat = xhat_fn(2)

  #Calculate RG
  RG = calculate_gap(x,xhat)

  #Print itterations
  print ("Itter:" ,k, "RG:", RG[0], "TSTT:", RG[1], "SPTT:", RG[2])

  #Update k
  k+=1
    

  


Itter: 1 RG: 85.87272206478525 TSTT: 531604867.788886 SPTT: 6119353.177312002
Itter: 2 RG: 14.706835352096958 TSTT: 156909219.26414225 SPTT: 9989868.471066255
Itter: 3 RG: 2.5397738555330687 TSTT: 56239025.32803832 SPTT: 15887745.269413281
Itter: 4 RG: 0.9874948831465671 TSTT: 30780486.19254417 SPTT: 15487076.94975951
Itter: 5 RG: 0.5411057063235749 TSTT: 22146142.390749544 SPTT: 14370294.19843033
Itter: 6 RG: 0.35825030895999843 TSTT: 17872453.320975896 SPTT: 13158438.619948257
Itter: 7 RG: 0.289361423485859 TSTT: 15520474.381441886 SPTT: 12037334.21733794
Itter: 8 RG: 0.24109876528597263 TSTT: 13982251.567842696 SPTT: 11266026.491147883
Itter: 9 RG: 0.21434462823066602 TSTT: 12945579.900165696 SPTT: 10660548.578394724
Itter: 10 RG: 0.18893203471466502 TSTT: 12170350.857125832 SPTT: 10236372.224629834
Itter: 11 RG: 0.17495592685613715 TSTT: 11588323.201831443 SPTT: 9862772.66828097
Itter: 12 RG: 0.15594788133014315 TSTT: 11127571.429161038 SPTT: 9626360.84972672
Itter: 13 RG: 0.145286

## **Print Results**


In [67]:
print("Number of Nodes:", NumNode)
print("\nNumber of Links:", Numlink)
print("\nNumber of Zones:", NumZone)
print("\nTime Taken:")
print(time.time() - start_time)
print("\n")
print("\nRG:", RG[0], "TSTT:", RG[1], "SPTT:", RG[2])



Number of Nodes: 24

Number of Links: 76

Number of Zones: 24

Time Taken:
8.067049980163574



RG: 0.0009991897102390457 TSTT: 7497384.342333955 SPTT: 7489900.510812836


In [68]:
print("Equilibrium Flows:")
print(x)

Equilibrium Flows:
[4511.784511784506, 8135.465768799097, 4536.02693602692, 5971.77328843993, 8111.2233445566735, 14055.274971941597, 10086.58810325476, 14077.104377104375, 18040.46015712678, 5216.217732884382, 18063.636363636346, 8798.540965207641, 15802.020202020174, 5996.0157126823815, 8806.790123456742, 12502.80583613915, 12116.04938271606, 15826.374859708152, 12535.297418630722, 12056.509539842868, 6889.337822671146, 8393.153759820423, 15816.947250280533, 6843.6588103254635, 21764.927048260313, 21834.175084175105, 17738.27160493824, 23144.50056116721, 11050.729517396157, 8109.2031425364685, 5314.870931537596, 17614.702581369194, 8370.370370370358, 9782.659932659908, 10040.516273849602, 8407.519640852972, 12342.648709315374, 12433.726150392753, 11124.523007856355, 9820.594837261488, 9041.975308641973, 8399.719416386071, 23209.708193041537, 9084.624017957312, 19097.13804713807, 18412.457912457852, 8411.78451178448, 11078.058361391686, 11692.592592592533, 15350.112233445549, 8109.483